In [ ]:
import os
import itertools
import time
import json
import tqdm

import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pathlib import Path
# %matplotlib inline

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.model_selection import KFold, train_test_split

# Feature scaling, required for non-tree-based models
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.stats.mstats import winsorize

# Encoding categorical data for non-tree-based models
from sklearn.preprocessing import OneHotEncoder

from fbprophet import Prophet

from tqdm import tqdm as tqdm

In [ ]:
!ls ../input

In [ ]:
goodsale = pd.read_csv('../input/goodsale.csv', thousands=',')
submitfile = pd.read_csv("../input/submit_example.csv")
# full_date = pd.read_csv("../input/full_date.csv")

In [ ]:
##格式改为date_time
goodsale["data_date"] = goodsale.data_date.map(lambda data:str(data))
goodsale["data_date"] = pd.to_datetime(goodsale.data_date)

In [ ]:
#Full Date Generate
full_date = goodsale[['data_date']] #筛选date
full_date = full_date.drop_duplicates() #去重
full_date = full_date.sort_values(by = 'data_date') #排序
full_date["data_date"] = pd.to_datetime(full_date.data_date)
full_date

In [ ]:
# %%time
#精简为11W 并
sim_goodsale = submitfile.merge(goodsale, on = 'sku_id')
sim_goodsale = sim_goodsale[['sku_id', 'data_date', 'goods_num']]

In [ ]:
sim_goodsale = sim_goodsale.sort_values(by = ['data_date'])

In [ ]:
sim_goodsale = sim_goodsale.rename(
    columns={'data_date': 'ds', 'goods_num': 'y'})
sim_goodsale

In [ ]:
#分组
#grouped_goodsale是分组并按date排序的全体390WDF
grouped_goodsale = sim_goodsale.groupby(sim_goodsale['sku_id'])

In [ ]:
full_date = full_date.rename(columns = {'data_date': 'ds'})

In [ ]:
def result_sum(item_forecast, sku_id):
    week1 = item_forecast[(item_forecast['ds'] >= '2018-05-01') & (item_forecast['ds'] <= '2018-05-07')]
    week1 = week1[['yhat']]
    week1 = week1.apply(sum)
    
    week2 = item_forecast[(item_forecast['ds'] >= '2018-05-08') & (item_forecast['ds'] <= '2018-05-14')]
    week2 = week2[['yhat']]
    week2 = week2.apply(sum)
    
    week3 = item_forecast[(item_forecast['ds'] >= '2018-05-15') & (item_forecast['ds'] <= '2018-05-21')]
    week3 = week3[['yhat']]
    week3 = week3.apply(sum)
    
    week4 = item_forecast[(item_forecast['ds'] >= '2018-05-22') & (item_forecast['ds'] <= '2018-05-28')]
    week4 = week4[['yhat']]
    week4 = week4.apply(sum)
    
    week5 = item_forecast[(item_forecast['ds'] >= '2018-05-29') & (item_forecast['ds'] <= '2018-06-04')]
    week5 = week5[['yhat']]
    week5 = week5.apply(sum)
    
    d = {'sku_id': [sku_id], 'week1': [week1[0]], 'week2': [week2[0]], 'week3': [week3[0]],
        'week4': [week4[0]], 'week5': [week5[0]]}
    result_df = pd.DataFrame(data=d)
    return result_df

In [ ]:
# %%time
st = 50000
batch_size = 12
# from joblib import Parallel, delayed
# import multiprocessing

# n_cores = multiprocessing.cpu_count()
# list_submit = list(submitfile['sku_id'])[5:7]
list_submit = list(submitfile['sku_id'])[st:st+batch_size]
final_df = pd.DataFrame(columns=['sku_id', 'week1', 'week2', 'week3', 'week4', 'week5'])
# forecasts = []
for item in tqdm(list_submit):
    item_df = grouped_goodsale.get_group(item).drop('sku_id', axis=1)
    item_full_time_df = full_date.merge(item_df, how = 'outer')
    item_full_time_df.fillna(0, inplace=True)

    model = Prophet(weekly_seasonality=True, yearly_seasonality=True)
    model.fit(item_full_time_df)
    future = model.make_future_dataframe(periods=80)
    forecast = model.predict(future)
    final_df = pd.concat([final_df, result_sum(forecast, item)])
#     return result_sum(forecast, item)

# forecasts = Parallel(n_jobs=n_cores)(delayed(run)(item) for item in tqdm(list_submit))
# for item in list_submit:
#     run(item)
#     final_df = pd.concat([final_df, run(item)])
    
# for forecast in tqdm(forecasts):
#     final_df = pd.concat([final_df, forecast])
csv_name = "final_df_{}_{}.csv".format(st, st+batch_size)
final_df.to_csv(csv_name, index = False)
final_df